In [8]:
import tensorflow as tf
import os

num_epochs = 2
batch_size = 32
learning_rate = 0.001
data_dir = 'D:\DL\dogs_cats1'
train_cats_dir = data_dir + '/train/cats/'
train_dogs_dir = data_dir + '/train/dogs/'
test_cats_dir = data_dir + '/valid/cats/'
test_dogs_dir = data_dir + '/valid/dogs/'
checkpoint_path = data_dir + "/models/cp-{epoch:04d}.ckpt"

def _decode_and_resize(filename, label):
    image_string = tf.io.read_file(filename)            # 读取原始文件
    image_decoded = tf.image.decode_jpeg(image_string)  # 解码JPEG图片
    image_resized = tf.image.resize(image_decoded, [256, 256]) / 255.0
    return image_resized, label

if __name__ == '__main__':
    # 构建训练数据集
    train_cat_filenames = tf.constant([train_cats_dir + filename for filename in os.listdir(train_cats_dir)])
    train_dog_filenames = tf.constant([train_dogs_dir + filename for filename in os.listdir(train_dogs_dir)])
    train_filenames = tf.concat([train_cat_filenames, train_dog_filenames], axis=-1)
    train_labels = tf.concat([
        tf.zeros(train_cat_filenames.shape, dtype=tf.int32), 
        tf.ones(train_dog_filenames.shape, dtype=tf.int32)], 
        axis=-1)

    train_dataset = tf.data.Dataset.from_tensor_slices((train_filenames, train_labels))
    train_dataset = train_dataset.map(
        map_func=_decode_and_resize, 
        num_parallel_calls=tf.data.experimental.AUTOTUNE)
    # 取出前buffer_size个数据放入buffer，并从其中随机采样，采样后的数据用后续数据替换
    train_dataset = train_dataset.shuffle(buffer_size=23000)    
    train_dataset = train_dataset.batch(batch_size)
    train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(256, 256, 3)),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(32, 5, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax')
    ])
     
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.sparse_categorical_crossentropy,
        metrics=[tf.keras.metrics.sparse_categorical_accuracy]
    )

    checkpoint_dir = os.path.dirname(checkpoint_path)

    # 创建一个保存模型权重的回调
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1, period=1)

    # 使用新的回调训练模型
    model.fit(train_dataset,
              epochs=num_epochs,
              callbacks=[cp_callback])  # 通过回调训练

    # 这可能会生成与保存优化程序状态相关的警告。
    # 这些警告（以及整个笔记本中的类似警告）是防止过时使用，可以忽略。
    
   

   # model.fit(train_dataset, epochs=num_epochs)

Epoch 1/2
     58/Unknown - 115s 2s/step - loss: 0.8393 - sparse_categorical_accuracy: 0.5417
Epoch 00001: saving model to D:\DL\dogs_cats1/models/cp-0001.ckpt
58/58 [==============================] - 116s 2s/step - loss: 0.8393 - sparse_categorical_accuracy: 0.5417
Epoch 2/2
57/58 [============================>.] - ETA: 1s - loss: 0.6550 - sparse_categorical_accuracy: 0.6146
Epoch 00002: saving model to D:\DL\dogs_cats1/models/cp-0002.ckpt
58/58 [==============================] - 112s 2s/step - loss: 0.6550 - sparse_categorical_accuracy: 0.6148


In [21]:
import tensorflow as tf
import os

img_path=r"D:\DL\dogs_cats1\test1\19.jpg"

img_raw = tf.io.read_file(img_path)            # 读取原始文件
img_tensor = tf.image.decode_jpeg(img_raw)  # 解码JPEG图片

print(img_tensor.shape)
print(img_tensor.dtype)
img_final = tf.image.resize(img_tensor, [256, 256]) / 255.0

print(img_final.shape)
print(img_final.numpy().min())
print(img_final.numpy().max())
image = tf.reshape(img_final, [-1, 256, 256, 3])


(272, 328, 3)
<dtype: 'uint8'>
(256, 256, 3)
0.023835784
1.0


In [22]:
model.predict(image)

array([[0.4464912, 0.5535088]], dtype=float32)

In [24]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'D:\\DL\\dogs_cats1/models\\cp-0002.ckpt'

In [25]:
#这一块是从最新的model 加载并使用
#create 新的model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(256, 256, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 5, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=[tf.keras.metrics.sparse_categorical_accuracy]
)
# 加载以前保存的权重
model.load_weights(latest)

 # 构建测试数据集
test_cat_filenames = tf.constant([test_cats_dir + filename for filename in os.listdir(test_cats_dir)])
test_dog_filenames = tf.constant([test_dogs_dir + filename for filename in os.listdir(test_dogs_dir)])
test_filenames = tf.concat([test_cat_filenames, test_dog_filenames], axis=-1)
test_labels = tf.concat([
     tf.zeros(test_cat_filenames.shape, dtype=tf.int32), 
     tf.ones(test_dog_filenames.shape, dtype=tf.int32)], 
     axis=-1)

test_dataset = tf.data.Dataset.from_tensor_slices((test_filenames, test_labels))
test_dataset = test_dataset.map(_decode_and_resize)
test_dataset = test_dataset.batch(batch_size)

print(model.metrics_names)
print(model.evaluate(test_dataset))

['loss', 'sparse_categorical_accuracy']
63/63 [==============================] - 40s 635ms/step - loss: 0.6608 - sparse_categorical_accuracy: 0.6200
[0.6608212826743959, 0.62]


In [26]:
model.save('my_model.h5')#保存在当天python的同目录下

In [27]:
# 重新创建完全相同的模型，包括其权重和优化程序
new_model = tf.keras.models.load_model('my_model.h5')

# 显示网络结构
new_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 254, 254, 32)      896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 123, 123, 32)      25632     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 61, 61, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 119072)            0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                7620672   
_________________________________________________________________
dense_5 (Dense)              (None, 2)                

In [28]:
new_model.fit(train_dataset, epochs=num_epochs)

Epoch 1/2
58/58 [==============================] - 112s 2s/step - loss: 0.5821 - sparse_categorical_accuracy: 0.6912
Epoch 2/2
58/58 [==============================] - 114s 2s/step - loss: 0.4034 - sparse_categorical_accuracy: 0.8298


In [ ]:
model.predict(image)

In [13]:
  # 构建测试数据集
test_cat_filenames = tf.constant([test_cats_dir + filename for filename in os.listdir(test_cats_dir)])
test_dog_filenames = tf.constant([test_dogs_dir + filename for filename in os.listdir(test_dogs_dir)])
test_filenames = tf.concat([test_cat_filenames, test_dog_filenames], axis=-1)
test_labels = tf.concat([
     tf.zeros(test_cat_filenames.shape, dtype=tf.int32), 
     tf.ones(test_dog_filenames.shape, dtype=tf.int32)], 
     axis=-1)

test_dataset = tf.data.Dataset.from_tensor_slices((test_filenames, test_labels))
test_dataset = test_dataset.map(_decode_and_resize)
test_dataset = test_dataset.batch(batch_size)

print(new_model.metrics_names)
print(new_model.evaluate(test_dataset))

['loss', 'sparse_categorical_accuracy']
63/63 [==============================] - 35s 557ms/step - loss: 0.6756 - sparse_categorical_accuracy: 0.5815
[0.6756408025347997, 0.5815]


In [7]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 254, 254, 32)      896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 123, 123, 32)      25632     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 61, 61, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 119072)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                7620672   
_________________________________________________________________
dense_3 (Dense)              (None, 2)                

In [3]:
  # 构建测试数据集
test_cat_filenames = tf.constant([test_cats_dir + filename for filename in os.listdir(test_cats_dir)])
test_dog_filenames = tf.constant([test_dogs_dir + filename for filename in os.listdir(test_dogs_dir)])
test_filenames = tf.concat([test_cat_filenames, test_dog_filenames], axis=-1)
test_labels = tf.concat([
     tf.zeros(test_cat_filenames.shape, dtype=tf.int32), 
     tf.ones(test_dog_filenames.shape, dtype=tf.int32)], 
     axis=-1)

test_dataset = tf.data.Dataset.from_tensor_slices((test_filenames, test_labels))
test_dataset = test_dataset.map(_decode_and_resize)
test_dataset = test_dataset.batch(batch_size)

print(model.metrics_names)
print(model.evaluate(test_dataset))

['loss', 'sparse_categorical_accuracy']
63/63 [==============================] - 39s 615ms/step - loss: 2.6947 - sparse_categorical_accuracy: 0.6070
[2.6946576549893333, 0.607]


In [ ]:
model.predict(test_dataset)

In [6]:
import tensorflow as tf
import os

img_path=r"D:\DL\dogs_cats1\test1\2.jpg"

img_raw = tf.io.read_file(img_path)            # 读取原始文件
img_tensor = tf.image.decode_jpeg(img_raw)  # 解码JPEG图片

print(img_tensor.shape)
print(img_tensor.dtype)
img_final = tf.image.resize(img_tensor, [256, 256]) / 255.0

print(img_final.shape)
print(img_final.numpy().min())
print(img_final.numpy().max())

(299, 296, 3)
<dtype: 'uint8'>
(256, 256, 3)
0.00054034067
1.0
